## 4 Kommissar Equi-Join's schwerster Fall (5 Punkte)
Copyright Jens Dittrich & Christian Schön, [Big Data Analytics Group](https://bigdata.uni-saarland.de/), [CC-BY-SA](https://creativecommons.org/licenses/by-sa/4.0/legalcode)

In this exercise you will help commissioner equi-join solving one of his old, unsolved cases.

## Requirements

This notebook is based on the sqlite3-kernel by Andrew Brownan which is [available on GitHub](https://github.com/brownan/sqlite3-kernel). As the kernel is based on the bash shell, it will only run on Unix systems, but not on Windows.

The easiest way to use this notebook is to use our `Vagrantfile` as explained [here](https://github.com/BigDataAnalyticsGroup/vagrant-bde).

Alternatively, if you want to install the sqlite kernel yourself, follow these steps:
1. Download the repository as zip file or clone it using git, if necessary unpack the archive.
2. If you are using a virtual machine, copy the folder to a location accessible by the virtual machine, e.g. a shared folder. Run your virtual machine and if necessary start your python environment.
3. Move to the folder containing the kernel and execute the following commands:
  - python setup.py install
  - python -m sqlite3_kernel.install
  
If the kernel was successfully installed, you should now be able to start jupyter notebook and select "Sqlite3" as notebook type for new notebooks. 

In case of problems, you can delete the kernel using the following steps:
1. Look up the kernel name by executing the command: jupyter kernelspec list
2. Delete the kernel: jupyter kernelspec uninstall kernel_name

## Load Data

Before we can start analyzing the data, we first have to load data from the corresponding csv files into an appropriate database schema. This is fake data.

In [1]:
PRAGMA foreign_keys = OFF;

DROP TABLE IF EXISTS purchases;
DROP TABLE IF EXISTS nutritionalValues;
DROP TABLE IF EXISTS livingIn;
DROP TABLE IF EXISTS households;
DROP TABLE IF EXISTS citizens;
DROP TABLE IF EXISTS articles;

PRAGMA foreign_keys = ON;

CREATE TABLE households (
    id INTEGER PRIMARY KEY,
    street TEXT,
    postcode INTEGER,
    city TEXT,
    floor INTEGER
);

CREATE TABLE citizens (
    id INTEGER PRIMARY KEY,
    firstname TEXT,
    lastname TEXT,
    birthday TEXT
);

CREATE TABLE livingIn (
    household_id INTEGER,
    citizen_id INTEGER,
    start TEXT,
    until TEXT,
    FOREIGN KEY(household_id) REFERENCES households(id),
    FOREIGN KEY(citizen_id) REFERENCES citizens(id),
    PRIMARY KEY(citizen_id, start, until)
);

CREATE TABLE articles (
    id INTEGER PRIMARY KEY,
    label TEXT,
    unit INTEGER
);

CREATE TABLE nutritionalValues (
    id INTEGER PRIMARY KEY,
    calories INTEGER,
    FOREIGN KEY(id) REFERENCES articles(id)
);

CREATE TABLE purchases (
    article_id INTEGER,
    citizen_id INTEGER,
    date TEXT,
    amount REAL,
    FOREIGN KEY(article_id) REFERENCES articles(id),
    FOREIGN KEY(citizen_id) REFERENCES citizens(id),
    PRIMARY KEY(article_id, citizen_id, date)
);

In [2]:
-- enable csv mode:
.mode csv

-- import the necessary files:
.import data/nsa/households_no_header.csv households
.import data/nsa/citizens_no_header.csv citizens
.import data/nsa/livingIn_no_header.csv livingIn
.import data/nsa/articles_no_header.csv articles
.import data/nsa/nutritionalValues_no_header.csv nutritionalValues
.import data/nsa/purchases_no_header.csv purchases

-- enable pretty formatting:
.mode columns
.headers on

### Your query

Enter your query in the following cell. It should output the list of main suspects in the following format:
1. The first names of the suspects as 'Vorname'
2. The last names of the suspects as 'Nachname'

You are allowed to use subqueries and views.

In [4]:
DROP VIEW IF EXISTS boughtPickle;

CREATE VIEW boughtPickle AS
    SELECT citizen_id
    FROM articles
        JOIN purchases
            ON id = article_id
    WHERE label = 'Pickle' AND amount = 0.5 AND '1943-11-19' <= date <= '1943-11-24';
    
DROP VIEW IF EXISTS boughtSalad;

CREATE VIEW boughtSalad AS
    SELECT citizen_id
    FROM articles
        JOIN purchases
            ON id = article_id
    WHERE label = 'Salad' AND amount >= 2 AND '1943-11-19' <= date <= '1943-11-24';
    
DROP VIEW IF EXISTS boughtCarrot;

CREATE VIEW boughtCarrot AS
    SELECT citizen_id
    FROM articles
        JOIN purchases
            ON id = article_id
    WHERE label = 'Carrot' AND 3 >= amount >= 1 AND '1943-11-19' <= date <= '1943-11-24';
    
DROP VIEW IF EXISTS address;

CREATE VIEW address AS
    SELECT citizen_id
    FROM households
        JOIN livingIn
            ON id = household_id
    WHERE street = '%18' AND street = '%straße 1%' AND street  = 'L%';
    
SELECT firstname AS Vorname, lastname AS Nachname
FROM boughtPickle AS p, boughtSalad AS s, boughtCarrot AS c, address AS a, citizens
WHERE p.citizen_id = s.citizen_id = c.citizen_id = a.citizen_id = id;